# Demo: Pronóstico de Demanda para Trámites Presenciales

Bienvenidos a la demo de IA predictiva en Red Hat OpenShift AI. Mostraremos cómo, a partir de datos históricos, podemos anticipar la demanda y tomar decisiones estratégicas en el sector público.

## Fase 1: Preparación del Entorno y Datos

En esta sección cargamos los datos históricos de la sucursal de atención ciudadana.

In [ ]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar datos
df = pd.read_csv('datos_sucursal.csv', parse_dates=['fecha'])

# Mostrar las primeras filas
df.head()

Como pueden ver, tenemos la fecha, el número de trámites y otra información relevante. Nuestro objetivo es usar estos datos para encontrar patrones.

## Fase 2: Visualización y Modelo de Pronóstico

Visualizamos la demanda histórica y entrenamos un modelo de pronóstico.

In [ ]:
# Visualización de la demanda histórica
plt.figure(figsize=(12,6))
sns.lineplot(x='fecha', y='tramites_atendidos', data=df, label='Histórico')
plt.title('Demanda histórica de trámites atendidos')
plt.xlabel('Fecha')
plt.ylabel('Trámites atendidos')
plt.legend()
plt.show()

Al visualizar los datos, podemos ver tendencias claras. Hay picos y valles recurrentes, y podemos inferir que la demanda varía según el día de la semana o si hay un día festivo.

In [ ]:
# Modelo de pronóstico simple usando Prophet
from prophet import Prophet

df_prophet = df.rename(columns={'fecha': 'ds', 'tramites_atendidos': 'y'})[['ds', 'y']]

model = Prophet()
model.fit(df_prophet)

# Generar predicción para las próximas 30 días
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Graficar resultados
plt.figure(figsize=(12,6))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Histórico', color='blue')
plt.plot(forecast['ds'], forecast['yhat'], label='Predicción', color='orange', linestyle='dashed')
plt.title('Pronóstico de demanda de trámites')
plt.xlabel('Fecha')
plt.ylabel('Trámites atendidos')
plt.legend()
plt.show()

La línea azul muestra los datos históricos. La línea punteada, que se proyecta hacia adelante, es la predicción de nuestro modelo para las próximas semanas.

## Fase 3: De la Predicción a la Acción Estratégica

Mostramos la tabla con los datos de la predicción y conectamos con un modelo LLM para recomendaciones.

In [ ]:
# Mostrar tabla de predicción para los próximos días
prediccion = forecast[['ds', 'yhat']].tail(10)
prediccion.rename(columns={'ds': 'fecha', 'yhat': 'tramites_predichos'}, inplace=True)
prediccion

El verdadero valor de la IA está en cómo usamos esta información para tomar decisiones: optimizar personal, mejorar el servicio y planificar recursos.

In [ ]:
# Conexión a LLM vía vLLM en OpenShift AI (con token)
import requests

ENDPOINT = "ENDPOINT/v1"
MODEL_NAME = "MODEL_NAME"
API_KEY = "API_KEY"

def obtener_recomendaciones(prompt):
    url = f"{ENDPOINT}/completions"
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": 200
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()["choices"][0]["text"]

prompt = f"Según la predicción, el próximo martes se esperan {int(prediccion.iloc[-1]['tramites_predichos'])} trámites. ¿Qué recomendaciones estratégicas puedes dar para optimizar la atención ciudadana ese día?"
recomendaciones = obtener_recomendaciones(prompt)
print(recomendaciones)


In [ ]:
# Pronóstico para una fecha específica o rango de fechas
from ipywidgets import widgets, interact
import pandas as pd
from datetime import datetime


# Widget para seleccionar rango de fechas
fecha_min = forecast['ds'].min()
fecha_max = forecast['ds'].max()
fecha_selector = widgets.SelectionRangeSlider(
    options=[str(d.date()) for d in pd.date_range(fecha_min, fecha_max)],
    index=(0, len(pd.date_range(fecha_min, fecha_max))-1),
    description='Rango:',
    orientation='horizontal',
    layout={'width': '800px'}
 )

@interact(rango=fecha_selector)
def mostrar_pronostico(rango):
    inicio, fin = rango
    df_rango = forecast[(forecast['ds'] >= pd.to_datetime(inicio)) & (forecast['ds'] <= pd.to_datetime(fin))][['ds', 'yhat']].copy()
    df_rango.rename(columns={'ds': 'fecha', 'yhat': 'tramites_predichos'}, inplace=True)
    display(df_rango)
    plt.figure(figsize=(12,6))
    plt.plot(df_rango['fecha'], df_rango['tramites_predichos'], label='Predicción', color='orange', linestyle='dashed')
    plt.title('Pronóstico de demanda para el rango seleccionado')
    plt.xlabel('Fecha')
    plt.ylabel('Trámites atendidos')
    plt.legend()
    plt.show()